<a href="https://colab.research.google.com/github/racoope70/daytrading-with-ml/blob/main/lightgbm_enhanced_strategy_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get remove --purge -y cuda* libcuda* nvidia* || echo "No conflicting CUDA packages"
!apt-get autoremove -y
!apt-get clean

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Note, selecting 'cuda-toolkit-12-4-config-common' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-11-0' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-11-1' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-11-7' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-11-8' for glob 'cuda*'
Note, selecting 'cuda-toolkit-12-5-config-common' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-12-0' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-12-1' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-12-2' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-12-3' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-12-4' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-12-5' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-12-6' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-12-8' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-12-9' for glob 'cuda*'
Note, selecting 'cuda-toolkit-12-6-config

In [2]:
#Protocol Buffer Fix (for TensorFlow)
!pip uninstall -y protobuf
!pip install protobuf==3.20.3

Found existing installation: protobuf 5.29.5
Uninstalling protobuf-5.29.5:
  Successfully uninstalled protobuf-5.29.5
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 11.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-status 1.71.0 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 3.20.3 which is incompatible.
ydf 0.12.0 requires protobuf<6.0.0,>=5.29.1, but you have protobuf 3.20.3 which is incompatible.
tensorflow-metadata 1.17.1 requires protobuf<6.0.0,>=4.25.2; python_version >= "3.11", but you have protobuf 3.20.3 which is incompatible.


In [3]:
#Update Colab Environment and System Libraries
!apt-get update -y && apt-get upgrade -y


Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:6 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:9 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [3,036 kB]
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,751 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packa

In [4]:
#Install Correct Version of CUDA for Colab GPU
!apt-get update -qq && apt-get install -y \
    libcusolver11 libcusparse11 libcurand10 libcufft10 libnppig10 libnppc10 libnppial10 \
    cuda-toolkit-12-4

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
E: Unable to locate package libnppig10
E: Unable to locate package libnppc10
E: Unable to locate package libnppial10


In [5]:
#Set Correct CUDA Paths
import os
os.environ['CUDA_HOME'] = '/usr/local/cuda-12.4'
os.environ['PATH'] += ':/usr/local/cuda-12.4/bin'
os.environ['LD_LIBRARY_PATH'] += ':/usr/local/cuda-12.4/lib64'


In [6]:
#Install RAPIDS and NVIDIA Dependencies
!pip install --extra-index-url=https://pypi.nvidia.com \
    cuml-cu12==25.2.0 cudf-cu12==25.2.0 cupy-cuda12x dask-cuda==25.2.0 dask-cudf-cu12==25.2.0


Looking in indexes: https://pypi.org/simple, https://pypi.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 127.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 219.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 86.7 MB/s eta 0:00:00
  Attempting uninstall: cudf-cu12
    Found existing installation: cudf-cu12 25.2.1
    Uninstalling cudf-cu12-25.2.1:
      Successfully uninstalled cudf-cu12-25.2.1
  Attempting uninstall: dask-cudf-cu12
    Found existing installation: dask-cudf-cu12 25.2.2
    Uninstalling dask-cudf-cu12-25.2.2:
      Successfully uninstalled dask-cudf-cu12-25.2.2
  Attempting uninstall: cuml-cu12
    Found existing installation: cuml-cu12 25.2.1
    Uninstalling cuml-cu12-25.2.1:
      Successfully uninstalled cuml-cu12-25.2.1


In [7]:
#Install TensorFlow (latest GPU-compatible version)
!pip install tensorflow==2.18.0

#Install Stable Baselines3 and Trading Libraries
!pip install stable-baselines3[extra] gymnasium gym-anytrading yfinance xgboost joblib

#Install Miscellaneous Libraries
!pip install matplotlib scikit-learn pandas numba==0.61.0

#Install PyTorch with GPU Support
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.2/172.2 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 119.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 92.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 43.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.5/184.5 kB 19.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-

In [8]:
#Install TensorFlow (latest GPU-compatible version)
!pip install tensorflow==2.18.0


import tensorflow as tf

gpus = tf.config.list_physical_devices("GPU")
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("TensorFlow GPU memory growth enabled")
    except RuntimeError as e:
        print(f"TensorFlow GPU memory config failed: {e}")


TensorFlow GPU memory growth enabled


In [9]:
!pip install stable-baselines3[extra] gymnasium gym-anytrading yfinance --quiet
!pip install stable-baselines3[extra] --quiet


In [10]:
!rm -rf /content/drive

In [11]:
from google.colab import drive
import os
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [12]:
#Add Live Updating / Online Learning
!pip install yfinance lightgbm scikit-learn matplotlib


In [13]:
#Noise Filtering
!pip install PyWavelets

In [14]:
pip install -U scikit-learn


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 117.9 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.6.1
    Uninstalling scikit-learn-1.6.1:
      Successfully uninstalled scikit-learn-1.6.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sklearn-compat 0.1.3 requires scikit-learn<1.7,>=1.2, but you have scikit-learn 1.7.0 which is incompatible.


In [1]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.9/395.9 kB 33.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.7/242.7 kB 25.7 MB/s eta 0:00:00


In [6]:
# === Imports and Warning Fixes ===
import os, gc, time, warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
import pywt
import optuna
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.model_selection import train_test_split
from lightgbm import LGBMClassifier, early_stopping, log_evaluation

warnings.filterwarnings("ignore", message=".*ensure_all_finite.*", category=FutureWarning)

# === Config ===
TEST_MODE = True
TICKERS = ['AAPL','TSLA', 'MSFT', 'GOOGL'] if TEST_MODE else [
    'AAPL', 'TSLA', 'MSFT', 'GOOGL', 'AMZN', 'NVDA', 'META', 'BRK-B', 'JPM', 'JNJ',
    'XOM', 'V', 'PG', 'UNH', 'MA', 'HD', 'LLY', 'MRK', 'PEP', 'KO', 'BAC', 'ABBV',
    'AVGO', 'COST', 'DIS', 'WMT', 'PFE', 'CSCO', 'TMO', 'ACN', 'CVX', 'ABT', 'CRM',
    'MCD', 'INTC', 'DHR', 'QCOM', 'TXN', 'NEE', 'AMGN', 'LIN', 'PM', 'BMY', 'MDT',
    'HON', 'UNP', 'LOW', 'ORCL', 'NKE', 'RTX', 'MS', 'GS']

PERIOD = "720d"
INTERVAL = "1h"
SAVE_DIR = "/content/drive/MyDrive/QuantConnect/results_lightgbm/lightgbm_walkforward_models"
RESULTS_DIR = "/content/drive/MyDrive/QuantConnect/results_lightgbm"
OUTPUT_PATH = f"{RESULTS_DIR}/multi_stock_feature_engineered_dataset.csv"
SLIPPAGE_RATE = 0.001
STOP_LOSS_PCT = 0.03
THRESHOLD = 0.55  # Can later be tuned per segment
FEATURES = ["SMA_50", "EMA_20", "RSI_voladj", "MACD_voladj", "Signal_Line", "ATR_voladj", "OBV", "CCI_voladj"]

os.makedirs(SAVE_DIR, exist_ok=True)
os.makedirs(os.path.join(RESULTS_DIR, "plots"), exist_ok=True)
os.makedirs(os.path.join(RESULTS_DIR, "data"), exist_ok=True)

# === Noise Filtering ===
def wavelet_denoise(series, wavelet="db1"):
    coeffs = pywt.wavedec(series, wavelet, mode="symmetric")
    coeffs[1:] = [np.zeros_like(c) for c in coeffs[1:]]
    return pywt.waverec(coeffs, wavelet, mode="symmetric")[:len(series)]

# === Regime Detection ===
def detect_regime(df, window=100):
    df["Return"] = df["Close"].pct_change()
    df["Vol"] = df["Close"].rolling(window).std()
    valid = df[["Return", "Vol"]].dropna()
    if len(valid) < 50:
        df["Regime"] = 1
        return df
    model = KMeans(n_clusters=3, random_state=42)
    labels = model.fit_predict(valid)
    regime = pd.Series(index=valid.index, data=labels)
    df["Regime"] = regime.reindex(df.index).ffill().bfill()
    return df

# === Feature Engineering ===
def compute_features(df):
    df['SMA_50'] = df['Close'].rolling(50).mean()
    df['EMA_20'] = df['Close'].ewm(span=20).mean()
    delta = df['Close'].diff()
    gain = delta.clip(lower=0).rolling(14).mean()
    loss = -delta.clip(upper=0).rolling(14).mean()
    rs = gain / (loss + 1e-6)
    df['RSI'] = 100 - (100 / (1 + rs))
    df['MACD'] = df['Close'].ewm(12).mean() - df['Close'].ewm(26).mean()
    df['Signal_Line'] = df['MACD'].ewm(9).mean()
    df['ATR'] = df['High'].rolling(14).max() - df['Low'].rolling(14).min()
    df['OBV'] = (np.sign(df['Close'].diff()) * df['Volume']).fillna(0).cumsum()
    tp = (df['High'] + df['Low'] + df['Close']) / 3
    df['CCI'] = (tp - tp.rolling(20).mean()) / (0.015 * tp.rolling(20).std())
    for col in ["RSI", "MACD", "CCI", "OBV", "ATR"]:
        df[f"{col}_voladj"] = wavelet_denoise(df[col].fillna(0).values)
    df = detect_regime(df)
    df['Target'] = (df['Close'].shift(-1) > df['Close']).astype(int)
    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    df.dropna(inplace=True)
    return df

# === Data Downloader ===
def download_data(ticker, retries=3):
    for attempt in range(retries):
        try:
            df = yf.download(ticker, period=PERIOD, interval=INTERVAL, progress=False)
            if not df.empty:
                df.reset_index(inplace=True)
                df['Symbol'] = ticker
                return df
        except Exception as e:
            print(f"Retry {attempt+1} for {ticker}: {e}")
            time.sleep((attempt + 1) * 5)
    print(f"Failed: {ticker}")
    return None

# === Optuna Tuner ===
def tune_lightgbm_with_optuna(X, y):
    def objective(trial):
        params = {
            "n_estimators": trial.suggest_int("n_estimators", 50, 300),
            "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
            "max_depth": trial.suggest_int("max_depth", 3, 10),
            "num_leaves": trial.suggest_int("num_leaves", 10, 150),
            "random_state": 42,
            "force_row_wise": True,
            "verbosity": -1,
        }
        model = LGBMClassifier(**params)
        model.fit(X_train, y_train)
        probas = model.predict_proba(X_val)[:, 1]
        return roc_auc_score(y_val, probas)

    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=25, show_progress_bar=False)
    return study.best_params

# === Walkforward with Tuning ===
def run_lgbm_walkforward(ticker, window_size=2000, step_size=250, initial_cash=100000):
    print(f"\n{ticker} | LGBM Walkforward")
    df = yf.download(ticker, period=PERIOD, interval=INTERVAL, auto_adjust=False, progress=False)
    if isinstance(df.columns, pd.MultiIndex): df.columns = df.columns.droplevel(1)
    df = compute_features(df)
    if df.empty or not all(col in df.columns for col in FEATURES): return

    scaler = MinMaxScaler()
    capital = initial_cash
    shares = capital / df['Close'].iloc[0]
    port_lgbm, port_hold, y_preds, y_true = [], [], [], []

    for start in range(0, len(df) - window_size, step_size):
        segment = df.iloc[start:start + window_size].copy()
        if segment.shape[0] < window_size: break
        X_all = scaler.fit_transform(segment[FEATURES])
        y_all = segment['Target'].values

        best_params = tune_lightgbm_with_optuna(X_all, y_all)
        model = LGBMClassifier(**best_params, random_state=42, force_row_wise=True)

        X_train, X_test = X_all[:-step_size], X_all[-step_size:]
        y_train, y_test = y_all[:-step_size], y_all[-step_size:]
        prices = segment['Close'].values[-step_size:]

        model.fit(X_train, y_train,
                  eval_set=[(X_test, y_test)],
                  callbacks=[early_stopping(20), log_evaluation(0)])

        probs = model.predict_proba(X_test)[:, 1]
        preds = (probs > THRESHOLD).astype(int)
        y_preds.extend(preds)
        y_true.extend(y_test.tolist())

        for i, pred in enumerate(preds):
            raw_price = prices[i]
            exec_price = raw_price * (1 + SLIPPAGE_RATE) if pred else raw_price * (1 - SLIPPAGE_RATE)
            if pred:
                if raw_price < exec_price * (1 - STOP_LOSS_PCT):
                    capital = shares * raw_price
                else:
                    capital = shares * exec_price
            else:
                shares = capital / exec_price
            port_lgbm.append(capital)
            port_hold.append(shares * raw_price)
        time.sleep(0.01)

    if len(y_preds) < 2: return
    y_preds, y_true = np.array(y_preds), np.array(y_true[:len(y_preds)])
    returns = np.diff(port_lgbm) / (np.array(port_lgbm[:-1]) + 1e-6)
    sharpe = np.mean(returns) / (np.std(returns) + 1e-6) * np.sqrt(252)
    drawdown = np.max(np.maximum.accumulate(port_lgbm) - port_lgbm)

    metrics = {
        "Ticker": ticker,
        "Final_Portfolio": round(port_lgbm[-1], 2),
        "Return_%": round((port_lgbm[-1] - initial_cash) / initial_cash * 100, 2),
        "Sharpe": round(sharpe, 4),
        "Accuracy": round(accuracy_score(y_true, y_preds), 4),
        "Precision": round(precision_score(y_true, y_preds), 4),
        "Recall": round(recall_score(y_true, y_preds), 4),
        "F1_Score": round(f1_score(y_true, y_preds), 4),
        "Drawdown": round(drawdown, 2)
    }

    pd.DataFrame([metrics]).to_csv(f"{RESULTS_DIR}/metrics_{ticker}.csv", index=False)
    model.booster_.save_model(f"{SAVE_DIR}/model_{ticker}.txt")

    plt.figure(figsize=(12, 6))
    plt.plot(port_lgbm, label="LGBM Strategy")
    plt.plot(port_hold, label="Buy & Hold")
    plt.title(f"{ticker} Portfolio Value")
    plt.legend()
    plt.tight_layout()
    plt.savefig(f"{RESULTS_DIR}/plots/{ticker}_portfolio_plot.png")
    plt.close()
    print(f"{ticker}: Return {metrics['Return_%']}%, Sharpe {metrics['Sharpe']}")

    summary_path = os.path.join(RESULTS_DIR, "lightgbm_walkforward_summary.csv")
    if os.path.exists(summary_path):
        all_df = pd.read_csv(summary_path)
        all_df = all_df[all_df["Ticker"] != ticker]
        all_df = pd.concat([all_df, pd.DataFrame([metrics])], ignore_index=True)
    else:
        all_df = pd.DataFrame([metrics])
    all_df.to_csv(summary_path, index=False)

# === Run All Tickers ===
for ticker in TICKERS:
    run_lgbm_walkforward(ticker)
    gc.collect()
    time.sleep(1)



AAPL | LGBM Walkforward


[I 2025-06-26 02:54:54,007] A new study created in memory with name: no-name-13432668-762d-4d07-96fc-ed0c5bcfb835
[I 2025-06-26 02:54:54,080] Trial 0 finished with value: 0.518532412965186 and parameters: {'n_estimators': 119, 'learning_rate': 0.13727789811293692, 'max_depth': 10, 'num_leaves': 34}. Best is trial 0 with value: 0.518532412965186.
[I 2025-06-26 02:54:54,151] Trial 1 finished with value: 0.48365596238495395 and parameters: {'n_estimators': 250, 'learning_rate': 0.04865654817348313, 'max_depth': 6, 'num_leaves': 119}. Best is trial 0 with value: 0.518532412965186.
[I 2025-06-26 02:54:54,259] Trial 2 finished with value: 0.5414790916366546 and parameters: {'n_estimators': 295, 'learning_rate': 0.17659235086568537, 'max_depth': 7, 'num_leaves': 57}. Best is trial 2 with value: 0.5414790916366546.
[I 2025-06-26 02:54:54,280] Trial 3 finished with value: 0.5066151460584234 and parameters: {'n_estimators': 57, 'learning_rate': 0.07835217500542414, 'max_depth': 4, 'num_leaves': 

Training until validation scores don't improve for 20 rounds
Early stopping, best iteration is:
[1]	valid_0's binary_logloss: 0.697159


[I 2025-06-26 02:54:56,424] Trial 3 finished with value: 0.5782877604166667 and parameters: {'n_estimators': 249, 'learning_rate': 0.22037534521605123, 'max_depth': 10, 'num_leaves': 112}. Best is trial 3 with value: 0.5782877604166667.
[I 2025-06-26 02:54:56,484] Trial 4 finished with value: 0.5391125801282051 and parameters: {'n_estimators': 121, 'learning_rate': 0.25339446138886246, 'max_depth': 5, 'num_leaves': 25}. Best is trial 3 with value: 0.5782877604166667.
[I 2025-06-26 02:54:56,646] Trial 5 finished with value: 0.5407902644230769 and parameters: {'n_estimators': 235, 'learning_rate': 0.1456035154328317, 'max_depth': 10, 'num_leaves': 131}. Best is trial 3 with value: 0.5782877604166667.
[I 2025-06-26 02:54:56,756] Trial 6 finished with value: 0.5415915464743589 and parameters: {'n_estimators': 294, 'learning_rate': 0.17902781454815805, 'max_depth': 4, 'num_leaves': 90}. Best is trial 3 with value: 0.5782877604166667.
[I 2025-06-26 02:54:56,780] Trial 7 finished with value: 

Training until validation scores don't improve for 20 rounds
Early stopping, best iteration is:
[1]	valid_0's binary_logloss: 0.699424


...

Training until validation scores don't improve for 20 rounds
Early stopping, best iteration is:
[7]	valid_0's binary_logloss: 0.693248


[I 2025-06-26 02:56:40,292] Trial 2 finished with value: 0.48636215905397634 and parameters: {'n_estimators': 272, 'learning_rate': 0.07107949772103307, 'max_depth': 6, 'num_leaves': 136}. Best is trial 1 with value: 0.4914497862446561.
[I 2025-06-26 02:56:40,365] Trial 3 finished with value: 0.4739993499837496 and parameters: {'n_estimators': 180, 'learning_rate': 0.04025252593286319, 'max_depth': 7, 'num_leaves': 116}. Best is trial 1 with value: 0.4914497862446561.
[I 2025-06-26 02:56:40,447] Trial 4 finished with value: 0.5025250631265781 and parameters: {'n_estimators': 233, 'learning_rate': 0.19821812754047358, 'max_depth': 9, 'num_leaves': 16}. Best is trial 4 with value: 0.5025250631265781.
[I 2025-06-26 02:56:40,479] Trial 5 finished with value: 0.46998674966874165 and parameters: {'n_estimators': 55, 'learning_rate': 0.09438929769593009, 'max_depth': 8, 'num_leaves': 31}. Best is trial 4 with value: 0.5025250631265781.
[I 2025-06-26 02:56:40,514] Trial 6 finished with value: 

Training until validation scores don't improve for 20 rounds
Early stopping, best iteration is:
[1]	valid_0's binary_logloss: 0.691889


[I 2025-06-26 02:56:42,440] Trial 2 finished with value: 0.46838670966774165 and parameters: {'n_estimators': 270, 'learning_rate': 0.21742750559353305, 'max_depth': 4, 'num_leaves': 113}. Best is trial 1 with value: 0.4893372334308358.
[I 2025-06-26 02:56:42,545] Trial 3 finished with value: 0.46454911372784313 and parameters: {'n_estimators': 216, 'learning_rate': 0.0467442157699261, 'max_depth': 10, 'num_leaves': 87}. Best is trial 1 with value: 0.4893372334308358.
[I 2025-06-26 02:56:42,643] Trial 4 finished with value: 0.49584989624740616 and parameters: {'n_estimators': 231, 'learning_rate': 0.2581223351994647, 'max_depth': 7, 'num_leaves': 46}. Best is trial 4 with value: 0.49584989624740616.
[I 2025-06-26 02:56:42,688] Trial 5 finished with value: 0.46148653716342913 and parameters: {'n_estimators': 93, 'learning_rate': 0.18113831609549544, 'max_depth': 8, 'num_leaves': 146}. Best is trial 4 with value: 0.49584989624740616.
[I 2025-06-26 02:56:42,739] Trial 6 finished with valu

Training until validation scores don't improve for 20 rounds
Early stopping, best iteration is:
[1]	valid_0's binary_logloss: 0.704159
GOOGL: Return -2.36%, Sharpe -0.0178


In [7]:
import pandas as pd

# Load the summary CSV file
summary_path = "/content/drive/MyDrive/QuantConnect/results_lightgbm/lightgbm_walkforward_summary.csv"
df = pd.read_csv(summary_path)

# Filter for AAPL
aapl_summary = df[df["Ticker"] == "AAPL"]
print(aapl_summary)


  Ticker  Final_Portfolio  Return_%  Sharpe  Accuracy  Precision  Recall  \
0   AAPL        113902.79      13.9  0.2136     0.489     0.5223  0.2026   

   F1_Score  Drawdown  
0    0.2919  20108.83  
